In [101]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchvision.transforms import transforms
import numpy as np
from PIL import Image

In [102]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [103]:
data = pd.read_csv("fmnist_small.csv")

In [104]:
x = data.drop('label', axis = 1).to_numpy()
y = data['label'].to_numpy()

In [105]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [106]:
custom_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [107]:
class CustomDataset(Dataset):
  def __init__(self, features, labels, transforms):
    self.features = features
    self.labels = labels
    self.transforms = transforms

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    image = self.features[index].reshape(28,28)
    image = image.astype(np.uint8)
    image = np.stack([image]*3, axis=-1)
    image = Image.fromarray(image)
    image_tensor = self.transforms(image)
    return image_tensor, torch.tensor(self.labels[index], dtype=torch.long)

In [108]:
train_dataset = CustomDataset(X_train, y_train, custom_transforms)
test_dataset = CustomDataset(X_test, y_test, custom_transforms)

In [109]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [110]:
from torchvision.models import vgg16
resnet_ashu = vgg16(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [111]:
resnet_ashu

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [112]:
for param in resnet_ashu.features.parameters():
  param.requires_grad = False

In [113]:
vgg16 = resnet_ashu.to(device)

In [114]:
learning_rate = 0.0001
epochs = 10

In [115]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [116]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_dataloader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = vgg16(batch_features)

    print(outputs.shape)
    print(batch_labels.shape)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

    break

  avg_loss = total_epoch_loss/len(train_dataloader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


torch.Size([32, 1000])
torch.Size([32])
Epoch: 1 , Loss: 0.09170134226481119
torch.Size([32, 1000])
torch.Size([32])
Epoch: 2 , Loss: 0.07067949295043946
torch.Size([32, 1000])
torch.Size([32])
Epoch: 3 , Loss: 0.04877884229024251
torch.Size([32, 1000])
torch.Size([32])
Epoch: 4 , Loss: 0.03869447072347005
torch.Size([32, 1000])
torch.Size([32])
Epoch: 5 , Loss: 0.033390687306722004
torch.Size([32, 1000])
torch.Size([32])
Epoch: 6 , Loss: 0.03371313412984212
torch.Size([32, 1000])
torch.Size([32])
Epoch: 7 , Loss: 0.02692631721496582
torch.Size([32, 1000])
torch.Size([32])
Epoch: 8 , Loss: 0.025159411430358887
torch.Size([32, 1000])
torch.Size([32])
Epoch: 9 , Loss: 0.01806175708770752
torch.Size([32, 1000])
torch.Size([32])
Epoch: 10 , Loss: 0.020676137606302897
